
# A template for analysing the data in python

This template is made to provide basic python commands for the reading and small processing of the data. Use it with care and try to adapt it to the spectra that you have taken during the lab. 

Implements:
-  file reading
-  simple peak finder that you will need to adapt maybe for each source
-  selection of a region around a peak

You will need to implement
-  the uncertainties on the y variable 
-  a fit of the data with a Gaus function
-  a linear fit to perform the energy calibration
-  determine the energy resolution dependence 
-  ...

Where to find information: internet is amazing for python


In [ ]:
from textwrap import dedent

def code_hider():
    """Make a button in the jupyter notebook to hide all code"""
    # Stolen from stackoverflow... forget which question
    # I would really like these buttons for every individual cell.. but I don't know how
    from IPython.display import HTML    
    return HTML(dedent('''
                       <script>
                       code_show=true
                       function code_toggle() {
                        if (code_show){
                        $('div.input').hide();
                          } else {
                        $('div.input').show();
                        }
                        code_show = !code_show
                       }
                       //$( document ).ready(code_toggle);
                       </script>
                       <form action="javascript:code_toggle()"><input type="submit"
                       value="Show/hide  all code in this notebook"></form>'''))

code_hider()


In [ ]:
# Boilerplate startup code
%matplotlib notebook
%matplotlib inline
import math
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import codecs
from datetime import datetime
from scipy.ndimage.filters import gaussian_filter

matplotlib.rc('font', size=16)
plt.rcParams['figure.figsize'] = (15.0, 10.0)    # resize plots


def read_file(file):
    ''' a simple function to read the files and produce a pandas DataFrame '''
    #    arguments: file = the file you want to read
    #      returns: df = pandas DataFrame
    
    COMMENT_CHAR = '#'
    columns = []
    with file as td:
        for line in td:
            # find the commented lines
            if line[0] == COMMENT_CHAR or line[0] =='t' :
                if ('t=') in line:
                    splitvalues= line.split(" ")
                    time = float(splitvalues[1][0:-2])
                    
            # when we see the first line that doesn't start with 
            # COMMENT_CHAR, we pass the remaining lines of the 
            # file to pandas.read_table and break our loop
            else:
                _dfs = [
                        pd.DataFrame([line.split()],  dtype=int),
                        pd.read_table(td, sep='\s+', names=['channel', 'N'], header=None, engine='python')
                        ]
                df = pd.concat(_dfs, ignore_index=True)
    
    df['NEr'] = [math.sqrt(y)/time for y in df.N]
    df['nu'] = df.N/time
    #remove rows if at least 3 values in a row are NaN
    df = df.dropna(thresh=3)
    # drop the 2 first columns
    df = df.drop([0,1], axis = 1)
    return df



In [ ]:
filename = 'Cs.txt'

fileCs = codecs.open(filename, encoding = "ISO-8859-1")
data = read_file(fileCs)

# plot something
plt.plot(data.channel, data.N, label="Your source")
plt.legend()
plt.ylabel("Y axis label [units]")
plt.xlabel("X axis label [units]")



In [ ]:
from scipy import signal
data = data.loc[data.channel<1100]
def find_peaks(data, do_the_plot = True):
    #function to find the peaks
    #   returns: the x positions of the peaks
    
    # change these parameters to be suit better your case
    widths = np.arange(15, 100, 20)
    peakind = signal.find_peaks_cwt(data.N, widths)
    if do_the_plot:
        plt.plot(data.channel, data.N, label="Your source")
        plt.plot(data.channel[peakind], data.N[peakind], color="red", linewidth=0, marker='o', label="peaks")
        plt.ylabel("Y axis label [units]")
        plt.xlabel("X axis label [units]")
        plt.legend()
    return peakind
find_peaks(data)

In [ ]:
def select_region(data, peak_value, nr_of_channels):
    # function to select part of the spectrum
    peak_data = data.loc[data.channel < peak_value + nr_of_channels]
    peak_data = peak_data.loc[data.channel > peak_value - nr_of_channels]
    return peak_data

interesting = select_region(data, 800, 100)

plt.plot(interesting.channel, interesting.N, label="Your source")
plt.legend()
plt.ylabel("Y axis label [units]")
plt.xlabel("X axis label [units]")

